In [171]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import chart_studio.plotly as py
import plotly.express as px
pd.set_option("display.max_columns", None)

### Read data
- Read the load data file
- Read the column decription file

In [316]:
data = pd.read_csv('../data/loan.csv', low_memory=False)
meta_data = pd.read_excel('../data/Data_Dictionary.xlsx')
meta_data.columns = ['column_name', 'description']

Getting the info of loan data using `data.info()` function

In [317]:
data.info(
    max_cols = 111,
    show_counts = True
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              39717 non-null  int64  
 1    member_id                       39717 non-null  int64  
 2    loan_amnt                       39717 non-null  int64  
 3    funded_amnt                     39717 non-null  int64  
 4    funded_amnt_inv                 39717 non-null  float64
 5    term                            39717 non-null  object 
 6    int_rate                        39717 non-null  object 
 7    installment                     39717 non-null  float64
 8    grade                           39717 non-null  object 
 9    sub_grade                       39717 non-null  object 
 10   emp_title                       37258 non-null  object 
 11   emp_length                      38642 non-null  object 
 12   home_ownership  

### View of loan data
Top 5 rows of load data 

In [318]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.0,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Column analysis

Create new data frame for colum analysis where computed following data for each column
- Total number of missing values
- Total % of missing value
- Merged with description data
- Data type of column
- A shample column which store
    - If column is of int or float type then min and max value
    - If column is of object type the few shample values
- New categorical column for missing value categories depends on % of missing values
    - If 0 then VL (very low)
    - If b/w 1 - 10 then L(low)
    - If b/w 11 - 30 then M(medium)
    - If b/w 31 - 80 then H(high)
    - If b/w 81 - 100 then VH(very high)

In [319]:
column_analysis = pd.DataFrame(
    {'missing_value_percent':data.apply(lambda x:round((x.isna().sum()/x.shape[0])*100,2)),
     'total_of_nan':data.isna().sum(),
    'column_name':data.columns}
)


In [321]:
column_analysis.head()

,missing_value_percent,total_of_nan,column_name
id,0.0,0,id
member_id,0.0,0,member_id
loan_amnt,0.0,0,loan_amnt
funded_amnt,0.0,0,funded_amnt
funded_amnt_inv,0.0,0,funded_amnt_inv


In [322]:
column_analysis.reset_index(inplace = True,drop = True)


In [323]:
column_analysis.head()

,missing_value_percent,total_of_nan,column_name
0,0.0,0,id
1,0.0,0,member_id
2,0.0,0,loan_amnt
3,0.0,0,funded_amnt
4,0.0,0,funded_amnt_inv


In [324]:
# Creating buckets according to percentage missing value
column_analysis['missing_category'] = pd.cut(x= column_analysis['missing_value_percent'], 
                                             bins= [-1.0, 0.0, 10.0, 30.0,80.0, 100.0],
                                            labels = ['Very Low (0% missing)','Low (1-10% missing)',
                                                  'Medium (11-30% missing)','High (31-80% missing)',
                                                  'Very High (80-100% missing)'])

In [325]:
def get_column_type(col_name):
    return data[col_name].dtype
column_analysis['dtype'] = column_analysis.column_name.apply(get_column_type)

In [326]:
def get_sample_value(col_name):
    col = column_analysis[column_analysis.column_name == col_name]
    dtype = col['dtype'].values[0]
    col_value = data[-data[col_name].isna()][col_name].values
    if (dtype == int or dtype == float) and col['missing_value_percent'].values[0] < 100:
        return F'{min(col_value)} | {max(col_value)}'
    elif dtype == object:
        _values = pd.Series(list(map(str,col_value))).unique()
        if(len(_values)/len(data) <= .15):
            return ' | '.join(_values)
        return ' | '.join(_values[:10])
    else:
        return None
column_analysis['sample_value'] = column_analysis.column_name.apply(get_sample_value)    

In [327]:
column_analysis.head()

,missing_value_percent,total_of_nan,column_name,missing_category,dtype,sample_value
0,0.0,0,id,Very Low (0% missing),int64,None
1,0.0,0,member_id,Very Low (0% missing),int64,None
2,0.0,0,loan_amnt,Very Low (0% missing),int64,None
3,0.0,0,funded_amnt,Very Low (0% missing),int64,None
4,0.0,0,funded_amnt_inv,Very Low (0% missing),float64,0.0 | 35000.0


### Calculated % of column under each category

Grouping rows according to missing_category column using pivot table
- create column from pivot table index 
- drop index
- plot pie chart

In [328]:
missing_category_percentage = pd.pivot_table(column_analysis,index = 'missing_category',values = 'missing_value_percent',aggfunc = lambda x:len(x)/len(column_analysis)*100)

In [329]:
missing_category_percentage['missing_category'] = missing_category_percentage.index


In [330]:
missing_category_percentage.reset_index(inplace = True,drop = True)

In [331]:
missing_category_percentage

,missing_value_percent,missing_category
0,38.738739,Very Low (0% missing)
1,9.009009,Low (1-10% missing)
2,1.801802,High (31-80% missing)
3,50.450450,Very High (80-100% missing)


### Ploting missing % of columns values category wise

Some intresting insite found
- 50% of columns have 100% missing value
- 1.8% of columns have 31 - 80% missing values
- Only 38.7% of columns have 0% missing values

So decided to go with only those rows which have less then 10% missing value

![Visualization](./newplot.png)

In [332]:
fig = px.pie(
    missing_category_percentage, 
    values='missing_value_percent', 
    names='missing_category',
    title='Percentage of column under mssing categories(depends on % on value missing)',
    hole=.5,
    color_discrete_sequence=px.colors.sequential.Agsunset_r,
    labels={'missing_category':'Category', 'column_percent': 'Column Percentage'},
)
fig.show()

In [ ]:
# column_analysis.set_index('column_name', inplace=True)

### Saving the column analysis data for future review

In [334]:
# column_analysis.to_excel('../data/explore_data.xlsx')
column_analysis.to_csv('../data/explore_data.csv')

In [335]:
missing_category_percentage

,missing_value_percent,missing_category
0,38.738739,Very Low (0% missing)
1,9.009009,Low (1-10% missing)
2,1.801802,High (31-80% missing)
3,50.450450,Very High (80-100% missing)


### dropping columns with more than 30 percent null values

In [337]:
column_values_with_more_than_30_percent_missing = list(column_analysis[column_analysis['missing_category'].isin(['High (31-80% missing)','Very High (80-100% missing)'])]['column_name'].values)

In [338]:
column_values_with_more_than_30_percent_missing

['desc',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'next_pymnt_d',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_pa

In [305]:
data_clean = data.drop(column_values_with_more_than_30_percent_missing,axis = 1)

In [315]:
data_clean.isna().sum()

id                               0
member_id                        0
loan_amnt                        0
funded_amnt                      0
funded_amnt_inv                  0
term                             0
int_rate                         0
installment                      0
grade                            0
sub_grade                        0
emp_title                     2459
emp_length                    1075
home_ownership                   0
annual_inc                       0
verification_status              0
issue_d                          0
loan_status                      0
pymnt_plan                       0
url                              0
purpose                          0
title                           11
zip_code                         0
addr_state                       0
dti                              0
delinq_2yrs                      0
earliest_cr_line                 0
inq_last_6mths                   0
open_acc                         0
pub_rec             

In [339]:
data_clean.to_csv('../data/nan_drop_data.csv')

In [340]:
data_clean.shape

(39717, 53)